In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import pprint, json
import requests
import datetime
import urllib.parse

# example URLs generated in https://github.com/evoactivity/PlutoIPTV
PLUTO_URL="http://api.pluto.tv/v2/channels?start=2022-02-20%2023%3A00%3A00.000%2B0100&stop=2022-02-22%2023%3A00%3A00.000%2B0100"
PLUTO_URL="http://api.pluto.tv/v2/channels?start=2022-02-28%2020%3A00%3A00.000%2B0100&stop=2022-03-02%2020%3A00%3A00.000%2B0100"
print(urllib.parse.unquote(PLUTO_URL))

In [ ]:
PLUTO_URL_TEMPLATE='http://api.pluto.tv/v2/channels?start={datetime_start}&stop={datetime_end}'
def create_pluto_url(dt_start, dt_end, template=PLUTO_URL_TEMPLATE):
    params = {"datetime_start": urllib.parse.quote(dt_start+".000+0100"),
              "datetime_end":   urllib.parse.quote(dt_end+".000+0100")}
    url=PLUTO_URL_TEMPLATE.format(**params)
    return url

def create_pluto_url_test():
    # test = create_pluto_url(dt_start="2022-02-20 23:00:00", dt_end="2022-02-22 23:00:00")
    test = create_pluto_url(dt_start="2022-02-28 20:00:00", dt_end="2022-03-02 20:00:00")
    print(PLUTO_URL)
    print(test)
    return test==PLUTO_URL

create_pluto_url_test()

In [ ]:
def pluto_url_around_now(pastdays=0, futuredays=2, time_of_day_string="20:00:00"):
    yesterday = datetime.datetime.now() - datetime.timedelta(days=pastdays)
    yesterday_string = ("%s" % yesterday)[:11] + time_of_day_string
    #print(yesterday_string)
    the_day_after_tomorrow = datetime.datetime.now() + datetime.timedelta(days=futuredays)
    tdat_string = ("%s" % the_day_after_tomorrow)[:11] + time_of_day_string
    #print(tdat_string)
    url = create_pluto_url(yesterday_string, tdat_string)
    return url

url = pluto_url_around_now()
print (url)
PLUTO_URL == url

In [ ]:
# url="http://api.pluto.tv/v2/channels?start=2022-02-27%2023%3A00%3A00.000%2B0100&stop=2022-03-01%2023%3A00%3A00.000%2B0100"
pluto=requests.get(url)
j=pluto.json()
if isinstance(j, dict) and j['statusCode']!=200:
    pprint.pprint(j)
else:
    print("Success. Result list has %d elements." % len(j))

In [ ]:
def extract_channel_slugs_only(j):
    slugs=[]
    for e in j:
        slugs.append(e['slug'])
    slugs=sorted(list(set(slugs)))
    return slugs

def print_with_newlines(j, func=extract_channel_slugs_only):
    print("\n".join(func(j)))

print(print_with_newlines(j))

In [ ]:
def extract_series_slugs_only(j):
    slugs=[]
    for ch in j:
        timelines = ch.get('timelines',[])
        for tl in timelines:
            ep=tl['episode']
            series_slug = ep['series']['slug']
            slugs.append(series_slug)
    slugs=sorted(list(set(slugs)))
    return slugs

print(print_with_newlines(j, extract_series_slugs_only))

In [ ]:
i=0
all,result=[],[]
for e in j:
    if my_channel_slugs =="*" or e['slug'] in my_channel_slugs:
        FORMATTER="(%s) %s = %d timelines"
        timelines = e.get('timelines',[])
        print (FORMATTER % (e['slug'], e['name'], len(timelines)))
        for tl in timelines:
            # pprint.pprint(tl); break
            ep=tl['episode']
            FORMATTER = "s%02de%02d"
            season = ep.get('season',0)
            exxsxx=FORMATTER % (season, ep['number'])
            start = datetime.datetime.strptime(tl['start'][:19],"%Y-%m-%dT%H:%M:%S")
            stop = datetime.datetime.strptime(tl['stop'][:19],"%Y-%m-%dT%H:%M:%S")
            minutes = int((stop-start).total_seconds() / 60.0)
            start_print = start.strftime("%H:%M %b %d")
            series_name = ep['series']['name']
            series_slug = ep['series']['slug']
            if "Star" not in series_name:
                break
            FORMATTER = "%s %s '%s' %s mins '%s' [%s] %s"
            value=FORMATTER % (exxsxx, start_print, ep['name'], minutes,
                               series_name, series_slug, e['slug'])
            key=e['slug']+"_%s"%start
            print(key)
            print(value)
            all.append((key,value))
            i+=1
        #break

if not result:
    print ("\n#############\nEMPTY results")
else:
    print("%s elements" % i)
    result = sorted(all, key=lambda x:x[0])
    print("\n".join(list(zip(*result))[1]))

In [ ]:
my_channel_slugs=['pluto-tv-sci-fi-de','pluto-tv-star-trek-de', 'doctor-who-classic-de']
# my_channel_slugs="*"
my_channel_slugs
my_series_slugs=['star-trek-discovery-de', 'star-trek-discovery-ptv2','star-trek-enterprise-de']
my_series_slugs=None

In [ ]:
def timelines_in_channel(ch, print_channel_name=True):
    timelines = ch.get('timelines',[])
    if print_channel_name:
        FORMATTER="(%s) %s = %d timelines"        
        print (FORMATTER % (ch['slug'], ch['name'], len(timelines)))
    return timelines

def exxsxx(ep):
    FORMATTER = "s%02de%02d"
    season = ep.get('season',0)
    return FORMATTER % (season, ep['number'])

def time_info(tl):
    start = datetime.datetime.strptime(tl['start'][:19],"%Y-%m-%dT%H:%M:%S")
    stop = datetime.datetime.strptime(tl['stop'][:19],"%Y-%m-%dT%H:%M:%S")
    minutes = int((stop-start).total_seconds() / 60.0)
    start_print = start.strftime("%H:%M %b %d")
    return start, start_print, minutes

def series_info(ep):
    series_name = ep['series']['name']
    series_slug = ep['series']['slug']
    return series_name, series_slug

def iterate_pluto(j, find_in_series_name=None, my_channel_slugs="*", print_channel_name=True, print_each_hit=False):
    hits, ch_slugs, series_slugs = [],[],[]
    for ch in j:
        if my_channel_slugs =="*" or ch['slug'] in my_channel_slugs:
            timelines = timelines_in_channel(ch, print_channel_name)
            for tl in timelines:
                start, start_print, minutes = time_info(tl)
                ep=tl['episode']
                series_name, series_slug = series_info(ep)
                
                if find_in_series_name and (find_in_series_name not in series_name):
                    break
                
                key=ch['slug']+"_%s"%start
                # print(key)
                
                FORMATTER = "%s %s '%s' %s mins '%s' [%s] %s"
                value=FORMATTER % (exxsxx(ep), start_print, ep['name'], minutes,
                                   series_name, series_slug, ch['slug'])
                if print_each_hit:
                    print(value)
                hits.append((key,value))
                ch_slugs.append(ch['slug'])
                series_slugs.append(series_slug)
            #break

    print("\n#################################\n%s elements.\n" % len(hits))
    result = sorted(hits, key=lambda x:x[0])
    return result, sorted(list(set(ch_slugs))), sorted(list(set(series_slugs)))

result, channels, series = iterate_pluto(j, print_channel_name=False, find_in_series_name="Star")
print("\n".join(list(zip(*result))[1]))
print ("Series: %s\nChannels: %s" % (series, channels))

In [ ]:
# careful 2840 answers:
result, channels, series = iterate_pluto(j, find_in_series_name=False, print_each_hit=False, print_channel_name=False)
print("\n".join(list(zip(*result))[1]))
print ("\n#################\nSeries: %s\n################\nChannels: %s" % (series, channels))

In [ ]:
# Everything with "Discovery":
result, channels, series = iterate_pluto(j, find_in_series_name="Star Trek", print_each_hit=False, print_channel_name=False)
if not result:
    print ("EMPTY result")
else:
    print("\n".join(list(zip(*result))[1]))
    print ("\n#################\nSeries: %s\n################\nChannels: %s" % (series, channels))

In [ ]:
# Everything with "Star":
result, channels, series = iterate_pluto(j, find_in_series_name="Star", print_each_hit=False, print_channel_name=False)
print("\n".join(list(zip(*result))[1]))
print ("#################\nSeries: %s\n################\nChannels: %s" % (series, channels))

In [ ]:
result, channels, series = iterate_pluto(j, find_in_series_name="Doctor Who", print_each_hit=False, print_channel_name=False)
print("\n".join(list(zip(*result))[1]))
print ("#################\nSeries: %s\n################\nChannels: %s" % (series, channels))